# BeamSearch for seq2seq model in keras that translates english to german

After [implementing Bytepairencoding](BytepairencodingForMachineTranslation.ipynb), I'll now optimize the decoding inference mechanism. Instead of always taking the most likely next symbol when decoding, BeamSearch keeps a candidate list of `beam_width` best translations so far and expands them all for the next symbol and takes from those new candidates again the best `beam_width` ones. I will implement it by scratch in python here. It would be an alternative to use the BeamSearch method from `tensorflow` (and probably use `tf.keras` instead of `Keras`). As my main purpose here is to go step by step on my own, I'll follow the educational approach to do it myself and use https://gist.github.com/udibr/67be473cf053d8c38730 as a template.

As trainings set I use the [European Parliament Proceedings Parallel Corpus 1996-2011](http://statmt.org/europarl/) German-English corpus with medium sized sentences.

I've refactored the code a bit (putting the seq2seq model and utils into own packages), so here we can focus on the beam search algorithm.

In [1]:
import gc
import os

from keras.backend.tensorflow_backend import set_session
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
 
import bytepairencoding as bpe
import seq2seq
from utils.download import download_and_extract_resources
from utils.linguistic import bleu_scores_europarl, read_europarl, preprocess_input_europarl as preprocess


# Fixing random state ensure reproducible results
RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
tf.set_random_seed(RANDOM_STATE)

pd.set_option('max_colwidth', 60)  # easier to read texts in e.g. df.head()

# technical detail so that an instance (maybe running in a different window)
# doesn't take all the GPU memory resulting in some strange error messages
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

/home/janek/.local/share/virtualenvs/rosetta-WKmHhL03/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
MAX_INPUT_LENGTH = 50
MAX_TARGET_LENGTH = 65
LATENT_DIM = 512
EMBEDDING_DIM = 300
BPE_MERGE_OPERATIONS = 5_000  # I'd love to use 10_000 x 300, but this one is broken: https://github.com/bheinzerling/bpemb/issues/6
EPOCHS = 20
BATCH_SIZE = 64
DROPOUT = 0.5
TEST_SIZE = 500
EMBEDDING_TRAINABLE = True  # Improves results significant and for at least it's not the most dominant training time factor (that's the output softmax layer)

## Download and explore data

In [3]:
PATH = 'data'
INPUT_LANG = 'en'
TARGET_LANG = 'de'
LANGUAGES = [INPUT_LANG, TARGET_LANG]
BPE_URL = {lang: f'http://cosyne.h-its.org/bpemb/data/{lang}/' for lang in LANGUAGES}
BPE_MODEL_NAME = {lang: f'{lang}.wiki.bpe.op{BPE_MERGE_OPERATIONS}.model' for lang in LANGUAGES}
BPE_WORD2VEC_NAME = {lang: f'{lang}.wiki.bpe.op{BPE_MERGE_OPERATIONS}.d{EMBEDDING_DIM}.w2v.bin' for lang in LANGUAGES}

EXTERNAL_RESOURCES = {
    # Europarl Corpus
    'de-en.tgz': 'http://statmt.org/europarl/v7/de-en.tgz',
    
    # Bytepairencoding subwords (_MODEL_) and pretrained embeddings (_WORD2VEC_)
    BPE_MODEL_NAME[INPUT_LANG]: f'{BPE_URL[INPUT_LANG]}/{BPE_MODEL_NAME[INPUT_LANG]}',
    BPE_WORD2VEC_NAME[INPUT_LANG] + '.tar.gz': f'{BPE_URL[INPUT_LANG]}/{BPE_WORD2VEC_NAME[INPUT_LANG]}' + '.tar.gz',
    BPE_MODEL_NAME[TARGET_LANG]: f'{BPE_URL[TARGET_LANG]}/{BPE_MODEL_NAME[TARGET_LANG]}',
    BPE_WORD2VEC_NAME[TARGET_LANG] + '.tar.gz': f'{BPE_URL[TARGET_LANG]}/{BPE_WORD2VEC_NAME[TARGET_LANG]}' + '.tar.gz',
    
    # Bytepairencoded model weights from BytepairencodingForMachineTranslation.ipynb
    'bytepairencoding_model_weights.h5': 'https://drive.google.com/open?id=1xK2QVTsIpJLmphSEUZl1Unqmz85MYeQK',
    'bytepairencoding_inference_encoder_model_weights.h5': 'https://drive.google.com/open?id=115Kp7ZIMqxu6YDvk4RhjvfYShcQdRP_o',
    'bytepairencoding_inference_decoder_model_weights.h5': 'https://drive.google.com/open?id=1_e3DE5lDw10joIb83UFbzGJyvQrMfb8w',
}

download_and_extract_resources(fnames_and_urls=EXTERNAL_RESOURCES, dest_path=PATH)

de-en.tgz already downloaded (188.6 MB)
en.wiki.bpe.op5000.model already downloaded (0.3 MB)
en.wiki.bpe.op5000.d300.w2v.bin.tar.gz already downloaded (6.2 MB)
de.wiki.bpe.op5000.model already downloaded (0.3 MB)
de.wiki.bpe.op5000.d300.w2v.bin.tar.gz already downloaded (5.7 MB)
bytepairencoding_model_weights.h5 already downloaded (31.5 MB)
bytepairencoding_inference_encoder_model_weights.h5 already downloaded (10.0 MB)
bytepairencoding_inference_decoder_model_weights.h5 already downloaded (21.5 MB)


In [4]:
df = pd.DataFrame(data={
    'input_texts': read_europarl(INPUT_LANG),
    'target_texts': read_europarl(TARGET_LANG)
})

In [5]:
print("Nr total input:", len(df))
df['input_length'] = df.input_texts.apply(len)
df['target_length'] = df.target_texts.apply(len)
df.head()

Nr total input: 1920209


,input_texts,target_texts,input_length,target_length
0,resumption of the session,wiederaufnahme der sitzungsperiode,25,34
1,i declare resumed the session of the european parliament...,"ich erkläre die am freitag, dem 0. dezember unterbrochen...",203,217
2,"although, as you will have seen, the dreaded 'millennium...","wie sie feststellen konnten, ist der gefürchtete ""millen...",191,185
3,you have requested a debate on this subject in the cours...,im parlament besteht der wunsch nach einer aussprache im...,105,110
4,"in the meantime, i should like to observe a minute' s si...",heute möchte ich sie bitten - das ist auch der wunsch ei...,232,217


### Filter translations (only sentences shorter than a given length)

With a full working machine translation system, it's of course better to train on all data (plus maybe some augmented data). Without attention (and maybe copy mechanism, dynamic memory, ...) there's no point anyway in it, but it also reduces training time (a full training on ~2 Mio translations might take days, even with a good GPU).
I use different length for input (english) than target (german) language as german is more verbose.

In [6]:
non_empty = (df.input_length > 1) & (df.target_length > 1)  # there are empty phrases like '\n' --> 'Frau Präsidentin\n'
short_inputs = (df.input_length < MAX_INPUT_LENGTH) & (df.target_length < MAX_TARGET_LENGTH)
print(f'Sentences with length between (1, input={MAX_INPUT_LENGTH}/target={MAX_TARGET_LENGTH}) characters:', sum(non_empty & short_inputs))
df = df[non_empty & short_inputs]
gc.collect();  # df with filtered sentences is significant smaller, so time to garbage collect

Sentences with length between (1, input=50/target=65) characters: 167211


## Load (pretrained) Bytepairs

I need the subwords dictionary (in `BPE_WORD2VEC_NAME`), the pretrained embeddings (in `BPE_MODEL_NAME`) and a [sentencepiece](https://github.com/google/sentencepiece) handler that can encode/decode them.

In [7]:
bpe_input, bpe_target = [bpe.Bytepairencoding(
    word2vec_fname=os.path.join(PATH, BPE_WORD2VEC_NAME[lang]),
    sentencepiece_fname=os.path.join(PATH, BPE_MODEL_NAME[lang]),
) for lang in [INPUT_LANG, TARGET_LANG]] 
print("English subwords", bpe_input.sentencepiece.EncodeAsPieces("this is a test for pretrained bytepairembeddings"))
print("German subwords", bpe_input.sentencepiece.EncodeAsPieces("das ist ein test für vortrainierte zeichengruppen"))

English subwords ['▁this', '▁is', '▁a', '▁test', '▁for', '▁pre', 'tr', 'ained', '▁by', 'te', 'pa', 'ire', 'm', 'bed', 'd', 'ings']
German subwords ['▁d', 'as', '▁is', 't', '▁e', 'in', '▁test', '▁f', 'ür', '▁v', 'ort', 'rain', 'ier', 'te', '▁ze', 'ic', 'hen', 'gr', 'up', 'p', 'en']


In [8]:
# Now encode the texts into sequences of indexes of bytepairs
df['input_sequences'] = df.input_texts.apply(bpe_input.subword_indices)
df['target_sequences'] = df.target_texts.apply(bpe_target.subword_indices)
df[['input_sequences', 'target_sequences']].head()

,input_sequences,target_sequences
0,"[1, 344, 146, 498, 90, 6, 3, 3235, 90, 2]","[1, 247, 351, 750, 5, 934, 43, 3158, 4762, 2]"
5,"[1, 3005, 416, 77, 359, 4, 241, 4, 17, 76, 451, 782, 21,...","[1, 241, 156, 72, 3112, 54, 4, 39, 26, 95, 4739, 89, 937..."
6,"[1, 29, 140, 414, 3231, 8, 3106, 2484, 9, 451, 782, 21, ...","[1, 35, 2444, 2269, 2109, 625, 39, 26, 95, 4739, 89, 937..."
7,"[1, 1599, 134, 546, 4, 19, 9, 918, 6, 535, 5, 2]","[1, 1161, 2266, 52, 4, 132, 2232, 1516, 3, 2]"
13,"[1, 1599, 134, 546, 4, 19, 9, 918, 6, 535, 5, 2]","[1, 1161, 2266, 52, 4, 132, 2232, 1516, 3, 2]"


In [9]:
# Those will be the inputs for the seq2seq model (that needs to know how long the sequences can get)
max_len_input = df.input_sequences.apply(len).max()
max_len_target = df.target_sequences.apply(len).max()

In [10]:
train_ids, val_ids = train_test_split(np.arange(df.shape[0]), test_size=0.1)

In [11]:
s2s = seq2seq.Seq2SeqWithBPE(
    bpe_input=bpe_input,
    bpe_target=bpe_target,
    max_len_input=max_len_input,
    max_len_target=max_len_target
)
s2s.model.load_weights(os.path.join(PATH, 'bytepairencoding_model_weights.h5'))
s2s.inference_encoder_model.load_weights(os.path.join(PATH, 'bytepairencoding_inference_encoder_model_weights.h5'))
s2s.inference_decoder_model.load_weights(os.path.join(PATH, 'bytepairencoding_inference_decoder_model_weights.h5'))

## Decode Beam Search implementation

In [12]:
def decode_beam_search(input_seq, beam_width):
    initial_states = s2s.inference_encoder_model.predict(input_seq)
    
    top_candidates = [{
        'states': initial_states,
        'idx_sequence': [bpe_target.start_token_idx],
        'token_sequence': [bpe_target.start_token],
        'score': 0.0,
        'live': True
    }]
    live_k = 1
    dead_k = 0
    
    for _ in range(max_len_target):
        if not(live_k and dead_k < beam_width):
            break
        new_candidates = []
        for candidate in top_candidates:
            if not candidate['live']:
                new_candidates.append(candidate)
                continue
         
            target_seq = np.zeros((1, max_len_target - 1))
            target_seq[0, 0] = candidate['idx_sequence'][-1]
            output, states = s2s.inference_decoder_model.predict(
                [target_seq, candidate['states']]
            )
            probs = output[0, 0, :]
        
            for idx in np.argsort(-probs)[:beam_width]:
                new_candidates.append({
                    'states': states,
                    'idx_sequence': candidate['idx_sequence'] + [idx],
                    'token_sequence': candidate['token_sequence'] + [bpe_target.tokens[idx]],
                    # sum -log(prob) numerical more stable than to multiplate probs                    
                    # goal now to minimize the score
                    'score': candidate['score'] - np.log(probs[idx]),  
                    'live': idx != bpe_target.stop_token_idx,
                })
        
        top_candidates = sorted(
            new_candidates, key=lambda c: c['score']
        )[:beam_width]
        
        alive = np.array([c['live'] for c in top_candidates])
        live_k = sum(alive == True)
        dead_k = sum(alive == False)
        
    return bpe_target.sentencepiece.DecodePieces(top_candidates[0]['token_sequence'])

In [13]:
def predict(sentence, beam_width=5):
    return decode_beam_search(pad_sequences(
        [bpe_input.subword_indices(preprocess(sentence))],
        padding='post',
        maxlen=max_len_input,
    ), beam_width=beam_width)

## Hyperparameter evaluation for beam_width and branch_width

In [14]:
val_ids, test_ids = val_ids[-TEST_SIZE:], val_ids[:TEST_SIZE]
print(len(train_ids), 'training size')
print(len(val_ids), 'validation size (hyperparam search)')
print(len(test_ids), 'test size (for final BLEU score)')

widths = [1, 2, 3, 5, 10, 20]
for beam_width in widths:
    bleu = bleu_scores_europarl(
        input_texts=df.input_texts.iloc[val_ids],
        target_texts=df.target_texts.iloc[val_ids],
        predict=lambda text: predict(text, beam_width=beam_width)
    )
    print(f'beam_width={beam_width} average BLEU = {bleu.mean()}')

150489 training size
500 validation size (hyperparam search)
500 test size (for final BLEU score)



beam_width=1 average BLEU = 0.30830853189276136



beam_width=2 average BLEU = 0.31319166873708537



beam_width=3 average BLEU = 0.3103411830919007



beam_width=5 average BLEU = 0.31256598297129945



beam_width=10 average BLEU = 0.31267283678076774



beam_width=20 average BLEU = 0.31135477696157376


In [15]:
# Performance on some examples:
EXAMPLES = [
    'Hello.',
    'You are welcome.',
    'How do you do?',
    'I hate mondays.',
    'I am a programmer.',
    'Data is the new oil.',
    'It could be worse.',
    "I am on top of it.",
    "N° Uno",
    "Awesome!",
    "Put your feet up!",
    "From the start till the end!",
    "From dusk till dawn.",
]
for en in [sentence + '\n' for sentence in EXAMPLES]:
    print(f"{preprocess(en)!r} --> {predict(en)!r}")

'hello.' --> 'hallam.'
'you are welcome.' --> 'seien sie willkommen.'
'how do you do?' --> 'was tun sie?'
'i hate mondays.' --> 'ich habe mir mißverständnisse.'
'i am a programmer.' --> 'ich bin ein programm.'
'data is the new oil.' --> 'daten sind die öls.'
'it could be worse.' --> 'das könnte besorgniserregend sein.'
'i am on top of it.' --> 'ich gehe davon aus.'
'n° uno' --> 'änderungsantrag 0'
'awesome!' --> 'einverstanden!'
'put your feet up!' --> 'fangen sie hart!'
'from the start till the end!' --> 'mit dem zielsetzungen!'
'from dusk till dawn.' --> 'von einem dialog.'


In [16]:
# Performance on training set:
for en, de in df[['input_texts', 'target_texts']][1:20].values.tolist():
    print(f"Original {en!r}, got {predict(en)!r}, exp: {de!r}")

Original "please rise, then, for this minute' s silence.", got 'bitte lassen sie mich das wort äußern.', exp: 'ich bitte sie, sich zu einer schweigeminute zu erheben.'
Original "(the house rose and observed a minute' s silence)", got '(das parlament erhebt sich zu einer schweigeminute.)', exp: '(das parlament erhebt sich zu einer schweigeminute.)'
Original 'madam president, on a point of order.', got 'frau präsidentin, zur geschäftsordnung.', exp: 'frau präsidentin, zur geschäftsordnung.'
Original 'madam president, on a point of order.', got 'frau präsidentin, zur geschäftsordnung.', exp: 'frau präsidentin, zur geschäftsordnung.'
Original 'thank you, mr segni, i shall do so gladly.', got 'vielen dank, herr segni.', exp: 'vielen dank, herr segni, das will ich gerne tun.'
Original 'it is the case of alexander nikitin.', got 'das ist der fall von alexander nikitin.', exp: 'das ist der fall von alexander nikitin.'
Original 'it will, i hope, be examined in a positive light.', got 'ich hoffe

In [17]:
# Performance on validation set
test_df = df.iloc[test_ids]
for en, de in test_df[['input_texts', 'target_texts']][1:20].values.tolist():
    print(f"Original {en!r}, got {predict(en)!r}, exp: {de!r}")

Original 'a lot of inspiration is needed.', got 'eine vielversprechnung ist erforderlich.', exp: 'diese denkanstöße sind dringend nötig.'
Original 'so what was cancún about?', got 'was ist also in cancún?', exp: 'worum ging es in cancún?'
Original 'i now turn to another subject.', got 'ich komme nun zu einem weiteren punkt.', exp: 'ich komme jetzt zu einem anderen punkt.'
Original 'thank you, mr prodi.', got 'vielen dank, herr prodi.', exp: 'vielen dank, herr prodi.'
Original 'the experience acquired varied somewhat.', got 'die erfahrungen haben etwas ähnliches.', exp: 'die damit gemachten erfahrungen sind recht unterschiedlich.'
Original 'that is what mr smith suggested.', got 'das hat smith gesagt.', exp: 'das hat herr smith vorgeschlagen.'
Original 'it is stable internally and externally.', got 'das ist in der tat exprimierend.', exp: 'er ist stabil nach innen und außen.'
Original 'mr president, i am wearing my irish scarf today.', got 'herr präsident, ich möchte meinen irischen bed

In [18]:
bleu = bleu_scores_europarl(
    input_texts=df.input_texts.iloc[test_ids],
    target_texts=df.target_texts.iloc[test_ids],
    predict=lambda text: predict(text)
)
print(f'average BLEU on test set = {bleu.mean()}')


average BLEU on test set = 0.32849784166287943


# Conclusion

The texts feel more readable, allthough the BLEU score rises up only a bit ($0.328 > $0.316$).
A lot of the problems in the translations certainly depend on the still small training set (~200k), so as next step, I'll train on a bigger sub-corpus of longer texts. This will also make the need to use an attention model more clear.